In [2]:
import pandas as pd
df = pd.read_csv('data_fin.csv', sep=';')

In [3]:
df.Movie_Id.nunique()

17770

In [4]:
df.Cust_Id.nunique()

480189

In [5]:
cust_sample=df.Cust_Id.sample(10000)
movie_sample=df.Movie_Id.sample(5000)

In [6]:
import surprise
from surprise import Reader, Dataset

In [ ]:
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(df[['Cust_Id','Movie_Id','Rating']],reader)

In [ ]:
from surprise import SVD

In [ ]:
#Задаём наш алгоритм и формируем обучающую выборку:

svd=SVD()
trainingSet=data.build_full_trainset()

In [ ]:
#Тренируем наш алгоритм:

svd.fit(trainingSet)

In [ ]:
#Далее мы уже можем получить предсказание:

testSet=trainingSet.build_testset()
predictions=svd.test(testSet)

In [ ]:
#Подгружаем названия фильмов:

titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [ ]:
#Далее можем задать функцию для определения топ-3 рекомендаций:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [ ]:
top3_recommendations = get_top3_recommendations(predictions)

In [ ]:
import numpy as np

def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

In [ ]:
i = 262149
print_recs(i)

In [ ]:
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)